In [6]:
import pandas as pd

filename = '../big_a_data/all.csv'
df = pd.read_csv(filename, converters={'code': str})
print(f'Done, {len(df)}')

Done, 5361


In [7]:
start = '2023-10-10'
mid = '2024-05-10'
end = '2024-07-12'

df['最低'] = 0.0 #start, mid
df['最高'] = 0.0 #start, end
df['最新'] = 0.0 #end
df['最高_比_最低'] = 0.0
df['最新_比_最高'] = 0.0

for idx_df, row_df in df.iterrows():
    code = row_df['code'] 
    print(code, end = '\r')
    file_daily = f'../big_a_data/hist_daily/{code}.csv'
    df_daily = pd.read_csv(file_daily)
    df_daily.set_index('日期', inplace=True)
    
    if (start not in df_daily.index or 
        mid   not in df_daily.index or 
        end   not in df_daily.index):
        continue

    df.at[idx_df, '最低'] = df_daily.loc[start:mid, '收盘'].min()
    df.at[idx_df, '最高'] = df_daily.loc[start:end, '收盘'].max()
    df.at[idx_df, '最新'] = df_daily.loc[end, '收盘']

df['最高_比_最低'] = round((df['最高']/df['最低'] - 1) * 100, 2)
df['最新_比_最高'] = round((df['最新']/df['最高'] - 1) * 100, 2)

print('\r\nDone', len(df))
df.head(3)

920002
Done 5361


,code,name,行业,上市时间,最新,涨幅,股息率_2023,最低,最高,最高_比_最低,最新_比_最高
0,000001,平安银行,银行,19910403,10.31,-0.1,7.1,9.03,11.56,28.02,-10.81
1,000002,万科A,房地产开发,19910129,6.95,2.41,0.0,6.54,12.84,96.33,-45.87
2,000004,国华网安,软件开发,19901201,9.96,1.81,0.0,9.12,19.52,114.04,-48.98


In [27]:
df_temp = df.copy()
df_temp['最新_比_最低'] = round((df['最新']/df['最低'] - 1) * 100, 2)
df_temp = df_temp[
                  (df_temp['最新_比_最高'] > -15) 
                  #(df_temp['最新_比_最低'] > -10) &
                  #(df_temp['最新_比_最低'] <  10)
                 ]
#df_temp.sort_values(by = ['最新_比_最高'], inplace = True)

df_temp = df_temp[df_temp['股息率_2023'] < 3]
df_temp.drop(['上市时间', '最低', '最高', '最新', '涨幅'], axis = 1, inplace = True)
df_temp.to_csv(f'股价波动_{start}_{mid}_{end}.csv', index = False)
print('Done', len(df_temp))
df_temp.head(3)

Done 404


,code,name,行业,股息率_2023,最高_比_最低,最新_比_最高,最新_比_最低
4,000007,全新好,房地产服务,0.00,65.61,-7.16,53.76
19,000027,深圳能源,电力行业,1.98,30.70,-7.74,20.58
40,000060,中金岭南,有色金属,1.26,36.86,-12.87,19.24
